![Banner](Banner.jpg)

# Definición de un MDP para Gridworld

En este tutorial vamos a definir un ambiente de prueba comúnmente utilizado en el aprendizaje por refuerzo; el ambiente de Gridworld. Vamos a definir el ambiente de Gridworld como un Markov Decision Process (MDP) para luego establecer solución con los diferentes métodos basados en modelos y libres de modelos.
Gridworld es un ambiente clásico de prueba dentro del aprendizaje por refuerzo. Durante este tutorial definiremos el modelo básico del ambiente, que extenderemos incrementalmente de acuerdo con las necesidades de los algoritmos específicos de solución que desarrollaremos a lo largo del curso.

Los ambientes de aprendizaje por refuerzo se pueden definir en dos (ambiente, agente) o tres (ambiente, agente, aprendizaje) módulos. Por simplicidad, dentro de nuestra implementación de Gridworld (e implementaciones posteriores) utilizaremos dos módulos. Sin embargo, cabe notar que una implementación utilizando tres módulos ofrece una mejor división de las características y por lo tanto una implementación más fácil de mantener y modificar.

Para cada uno de los módulos a continuación daremos una clase principal de código sobre la cual se deben hacer todas las modificaciones. Cada una de las funciones a implementar se presentarán progresivamente. Para cada una de las funciones a implementar se dará un conjunto (mínimo) de prueba.


## Ambiente 🌎

El ambiente de Gridworld se define como una cuadricula de `nxm`. El ambiente tiene obstáculos, es decir casillas por las cuales no puede pasar el agente. Al chocar con un obstáculo, el agente termina en la misma casilla en la que estaba. Un comportamiento similar se observa cuando el agente trata de salir de los bordes del ambiente. Además, el ambiente tiene una casilla de inicio definida, y algunas casillas de salida (determinadas por la recompensa asociada a ellas). Un ejemplo del ambiente para el caso `3x4` se muestra a continuación.

![gridworld.png](gridworld.png)

En este ejemplo del ambiente la casilla de inicio es la casilla inferior izquierda y tiene como objetivo llegar a la casilla de salida con recompensa 1. La otra casilla de salida tiene recompensa -1.


#### ¿Cómo podemos codificar el ambiente?

La definición del ambiente de Gridworld esta definida por:
1. Una cuadrícula rectangular (`grid`), con dimensiones `(n,m)` dadas por parámetro, donde la casilla superior izquierda esta en la posición (0,0). Definiremos las casillas por las que puede pasar el agente como espacios en blanco y las casillas por las que no puede pasar el agente con un `'#'`.
2. Las recompensas de cada casilla de la cuadrícula estan definidas dentro de la definición de `grid`.
    - +1 para la casilla objetivo 
    - -1 para las casillas de trampa
3. Un atributo con el estado actual (`state`) en el que se encuentra el agente. Por defecto este estado será la posición marcada como `S`.

Un ejemplo de una cuadrícula de 3x4, como se mostró anteriormente, sería así:

 ```python  
    board = [[' ', ' ', ' ',  +1],
            [' ', '#', ' ',  -1],
             ['S', ' ', ' ', ' ']] 
```

La definición del ambiente un ejemplo de recompensas se vería así: 

```python
    def __init__(self, board):
        # layout
        self.grid = copy_elements_from_the_board
```


#### 1. Estructura del ambiente

Defina la clase `Gridworld`, que recibe una cuadrícula con la descripción del tablero `board`, definidos como en el ejemplo. La información codificada en la cuadrícula será almacenada en el atributo `grid` del ambiente (este atributo corresponde al mdp donde se almacena la información del ambiente, la función de transición y las recompensas). Adicionalmente, para facilitar el uso del mdp, definimos los atributos para guardar la información de la cuadrícula; las filas (`nrows`) y columnas (`ncols`). esta información se da por parámetro (`dimensions`), al instanciar la clase, como una tupla con los valores respectivos. Adicionalmente la clase debe tener los atributos `initial_state` y `state` que corresponden al estado inicial y el estado actual del agente en el ambiente, respectivamente. Estos atributos se guardan como tuplas.

Finalmente, el atributo `grid` se debe codificar de tal forma que las casillas prohibidas (marcadas como `'#'`) no deben tener ningún valor asignado (su valor debe ser `None`), las casillas en blanco deben tener valor `0`, y las casillas con recompensas asociadas deben tener el valor de la recompensa como su valor. Note que con esta codificación de `grid`, y teniendo en cuenta que las acciones son determinísticas, estamos codificando directamente las recompensas del ambiente. Si este no fuera el caso, sería necesario definir un nuevo atributo `rewards`.

In [32]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

class Gridworld:
    def __init__(self, board: list[list[str]], dimensions: tuple[int,int]):
        self.nrows, self.ncols = dimensions
        self.grid = [[None for _ in range(self.ncols)] for _ in range(self.nrows)]
        self.initial_state = None
        self.state = None
        self.board = board

        for i in range(self.nrows):
            for j in range(self.ncols):
                cell = board[i][j]

                if cell == '#':
                    self.grid[i][j] = None
                elif cell == ' ':
                    self.grid[i][j] = 0
                elif cell == 'S':
                    self.initial_state = (i, j)
                    self.state = (i, j)
                    self.grid[i][j] = 0
                else:
                    try:
                        self.grid[i][j] = int(cell)
                    except:
                        self.grid[i][j] = cell

    def get_current_state(self) -> tuple[int,int]:
        return self.state

    def get_possible_actions(self, state:tuple[int,int]) -> list[str]:
        pass

    def do_action(self, action: str) -> tuple[float, tuple[int,int]]:
        pass

    def reset(self):
        self.state = self.initial_state
        return self.state

    def is_terminal(self) -> bool:
        i, j = self.state
        value = self.grid[i][j]
        return value == 1 or value == -1

    def get_action_index(self, action:str) -> int:
        actions = ['up', 'right', 'down', 'left']
        for idx, a in enumerate(actions):
            if a == action:
                return idx

    def plot(self):
        fig1 = plt.figure(figsize=(10, 10))
        ax1 = fig1.add_subplot(111, aspect='equal')

        for i in range(0, len(self.grid)+1):
            ax1.axhline(i , linewidth=2, color="#2D2D33")
            ax1.axvline(i , linewidth=2, color="#2D2D33")

        (i,j)  = self.initial_state
        ax1.add_patch(patches.Rectangle((j, self.nrows - i -1), 1, 1, facecolor = "#F6D924"))

        for j in range(len(self.grid[0])):
            for i in range(len(self.grid)):
                if self.grid[i][j] == 1:
                    ax1.add_patch(patches.Rectangle((j,self.nrows - i -1), 1, 1, facecolor = "#68FF33"))
                if self.grid[i][j] == None:
                    ax1.add_patch(patches.Rectangle((j,self.nrows - i -1), 1, 1, facecolor = "#6c7780"))
                if self.grid[i][j] == -1:
                    ax1.add_patch(patches.Rectangle((j,self.nrows - i -1), 1, 1, facecolor = "#cc0000"))

        plt.scatter(self.state[1] + 0.5, self.nrows - self.state[0] - 1 + 0.5, 
                    s=100, color="blue", marker="o", zorder=10)
        
        for i in range(len(self.grid)):
            for j in range(len(self.grid[0])):
                if self.grid[i][j] is not None:
                    ax1.text(j+0.5, self.nrows-i-1+0.5, self.grid[i][j], ha='center', va='center')

        plt.axis("off")
        plt.show()


In [33]:
#Pruebas estructura del ambiente
### BEGIN TESTS
board = [[' ', ' ', '-1'],
      ['S', '#', ' '],
      [' ', ' ', '+1']]

grid = Gridworld(board, (3,3))

try:
    grid.nrows
except:
    print("El atributo nrows no está definido")
try:
    grid.ncols
except:
    print("El atributo ncols no está definido")
try:
    grid.initial_state
except:
    print("El atributo initial_state no está definido")

try:
    grid.state
except:
    print("El atributo state no está definido")

try:
    grid.grid
except:
    print("El atributo grid no está definido")

assert grid.ncols == 3, "El valor de las columnas debe ser el dado por parámetro en el atributo dimensions"
assert grid.nrows == 3, "El valor de las columnas debe ser el dado por parámetro en el atributo dimensions"
assert grid.grid[1][1] == None, "El valor de las casillas sobre las que no puede pasar el agente debe ser None"
assert grid.grid[0][2] == -1, "El valor de las casillas con recompensa debe ser el valor numérico (tipo int)"
assert grid.initial_state == (1,0), "La posición inicial del agente debe ser la definida en el estado S de la cuadrícula dada"
assert grid.initial_state == grid.state, "La posición del agente debe ser igual a su posición inicial si ningún paso se ha dado"
### END TESTS

In [34]:
#Pruebas adicionales


#### 2. Movimientos del agente

Defina la función `do_action` que ejecuta la acción tomada por el agente dentro de la cuadrícula. Esta función recibe como parámetro la acción a ejecutar y retorna el valor de la casilla de llegada de la acción y el estado de llegada de la acción (como una tupla). Note que los movimientos fuera del tablero o las casillas prohibidas no deberían tener ningún efecto en la posición del agente (el agente se debe mantener en la misma posición de partida).

En esta versión de gridworld vamos a trabajar con acciones determinísticas, es decir el movimiento deseado del agente (`up`,`right`,`down`,`left`) siempre resultara en el estado esperado. 


In [35]:
class Gridworld(Gridworld):
    def do_action(self, action: str) -> tuple[float, tuple[int,int]]:
        i, j = self.state
        next_i, next_j = i, j

        if action == 'up':
            next_i -= 1
        elif action == 'down':
            next_i += 1
        elif action == 'left':
            next_j -= 1
        elif action == 'right':
            next_j += 1

        # Validar límites y obstáculos
        if next_i < 0 or next_i >= self.nrows or next_j < 0 or next_j >= self.ncols:
            next_i, next_j = i, j
        elif self.grid[next_i][next_j] is None:
            next_i, next_j = i, j

        self.state = (next_i, next_j)
        reward = self.grid[next_i][next_j]

        return reward, self.state
        

In [36]:
#Pruebas de movimiento
### BEGIN TESTS
board = [[' ', ' ', '-1'],
      ['S', '#', ' '],
      [' ', ' ', '+1']]

grid = Gridworld(board, (3,3))

try:
    grid.do_action
except:
    print("La función do_action no está definida")

val, state = grid.do_action('up')
assert state == (0,0), f"El movimiento hacia arriba no esta modificando correctamente al agente, se movió a {state} y debería ser (0,0)"
val, state = grid.do_action('right')
assert state == (0,1), f"El movimiento hacia la derecha no esta modificando correctamente al agente, se movió a {state} y debería ser (0,1)"
val, state = grid.do_action('right')
assert val == -1, f"El valor de las casillas no se esta retornando correctamente"
val, state = grid.do_action('down')
assert state == (1,2), f"El movimiento hacia la derecha no esta modificando correctamente al agente, se movió a {state} y debería ser (1,2)"
val, state = grid.do_action('left')
assert state == (1,2), f"El movimiento hacia la derecha no esta modificando correctamente al agente, se movió a {state} y debería ser (1,2) porque hay un obstáculo en (1,1)"

### END TESTS

In [37]:
#Pruebas adicionales



#### 3. Estado actual

Defina la función `get_current_state` que retorna el estado actual del agente (la casilla donde se encuentra el agente). Esta función no recibe ningún parámetro y retorna el estado actual como una tupla.

In [38]:
#Pruebas estado actual
### BEGIN TESTS
board = [[' ', ' ', '-1'],
      ['S', '#', ' '],
      [' ', ' ', '+1']]

grid = Gridworld(board, (3,3))

try:
    grid.get_current_state
except:
    print("La función get_current_state no está definida")

assert grid.get_current_state() == (1,0), "La posición del agente debe ser la posición inicial si no se realiza ningún movimeinto"
grid.do_action('up')
assert grid.get_current_state() == (0,0), f"La posición del agente no se esta modificando correctamente, recibió {grid.get_current_state()} cuando debería ser (0,0), revise los movimientos"
grid.state = (1,0)
grid.do_action('right')
assert grid.get_current_state() == (1,0), "El agente no se mantiene en la misma posición cuando trata de moverse a la casilla prohibida"
### END TESTS

In [39]:
#Pruebas adicionales



#### 4. Obtener las acciones

Defina la función `get_possible_actions` que recibe el estado actual del agente por parámetro y retorna una lista de las acciones válidas para el estado dado.

Tenga en cuenta que en esta versión de Gridworld, todas las acciones (i.e., los movimientos en las cuatro direcciones) son posibles para el agente en cada una de las casillas regulares de la cuadrícula. Las casillas de salida tienen única acción posible `'exit'` y las casillas prohibidas no tienen ningúna acción asociada (una lista vacía de acciones).

In [45]:
class Gridworld(Gridworld):
    def get_possible_actions(self, state):
        i, j = state
        cell = self.board[i][j]

        # Casilla bloqueada
        if cell == '#':
            return []

        # Casilla terminal
        if cell in ['-1', '+1']:
            return ['exit']

        # Casilla normal o inicial
        return ['up', 'right', 'down', 'left']

In [46]:
#Pruebas de acciones posibles
### BEGIN TESTS
board = [[' ', ' ', '-1'],
      ['S', '#', ' '],
      [' ', ' ', '+1']]

grid = Gridworld(board, (3,3))

try:
    grid.get_possible_actions
except:
    print("La función get_possible_actions no está definida")

a1 = grid.get_possible_actions(grid.state)
a2 = grid.get_possible_actions((1,0))
a3 = grid.get_possible_actions((1,2))
a4 = grid.get_possible_actions((0,1))
a5 = grid.get_possible_actions((2,1))
ap = grid.get_possible_actions((1,1))
ar1 = grid.get_possible_actions((0,2))
ar2 = grid.get_possible_actions((2,2))
assert ap == [], "Las acciones para las casillas prohibidas deben estar vacias"
assert a1 == a2 == a4 == a5, f"Todas las casillas regulares deben tener el mismo conjunto de acciones {['up', 'right', 'down', 'left']}"
assert ar1 == ar2, f"Todas las casillas de salida deben tener el mismo conjunto de acciones {['exit']}"
### END TESTS

In [47]:
#Pruebas adicionales




#### 5. Reinicializar el ambiente

Defina la función `reset` que no recibe parámetros ni retorna ningún valor. El efecto de esta función es restablecer el ambiente a su estado inicial.



In [48]:
#Pruebas de reset
### BEGIN TESTS
board = [[' ', ' ', '-1'],
      ['S', '#', ' '],
      [' ', ' ', '+1']]

grid = Gridworld(board, (3,3))

try:
    grid.reset
except:
    print("La función reset no está definida")

grid.state = (2,2)
grid.reset()
assert grid.get_current_state() == grid.initial_state, f"No se esta retornando al estado inicial. {grid.get_current_state()} y no {grid.initial_state}"
grid.state = (1,1)
grid.reset()
assert grid.get_current_state() == grid.initial_state, f"No se esta retornando al estado inicial. {grid.get_current_state()} y no {grid.initial_state}"
### END TESTS

In [49]:
#Pruebas adicionales




#### 6. Estados terminales

Defina la función `is_terminal` que determina si el agente está en un estado final o no. En nuestro caso los estados finales o los estados de salida estarán determinados por las casillas con recompensa 1 o -1.
Esta función no recibe parámetros y retorna un booleano determinando si el agente está en un estado final o no.

In [50]:
#Pruebas terminación
### BEGIN TESTS
board = [[' ', ' ', '-1'],
      ['S', '#', ' '],
      [' ', ' ', '+1']]

grid = Gridworld(board, (3,3))

try:
    grid.is_terminal
except:
    print("La función is_terminal no está definida")

grid.state = (2,2)
assert grid.is_terminal() == True, f"No se identifican correctamente los estados de salida como finales"
grid.state = (0,2)
assert grid.is_terminal() == True, f"No se identifican correctamente los estados de salida como finales"

for i in range(grid.nrows):
    for j in range(grid.ncols):
        grid.state = (i,j)
        if (i,j) == (0,2) or (i,j) == (2,2):
            assert grid.is_terminal() == True, f"No se identifican correctamente los estados de salida como finales"
        else:
            assert grid.is_terminal() == False, f"La casilla {(i,j)} se estan identificando como estado final"

### END TESTS

In [51]:
#Pruebas adicionales



## Preguntas de reflexión

Considere las siguientes situaciones y piense cual debería ser el resultado del MDP.

1. Plantee el problema de MDP para cada una de las casillas. Especifique el estado de inicio, las transiciones y su probabilidad. Para cada acción suponga una probabilidad de éxito de 0.25 (el 0.75 restante se resuelve uniformemente entre las acciones restantes).
¿Cómo serían las recompensas esperadas para cada estado?

2. Bajo la definción del problema anterior, suponga que ahora cada acción tiene una probabilidad de éxito de 60%, con probabilidad de 30% que se ejecutará la sigiente acción (en dirección de las manesillas del reloj) y con probabilidad de 10% que no se ejecute ninguna acción (el agente se queda quieto). Bajo estas condiciones, ¿Cómo serían las recompensas esperadas para cada estado? 
